In [2]:
import librosa
import os
import numpy as np

from dtw import dtw
from numpy.linalg import norm
from numpy import array
import pyaudio
import wave

import heapq

In [3]:
def initialCorpus(path):
    # 音乐库位置
    audioList = os.listdir(path)

    raw_audioList = {}
    beat_database = {}

    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav'):
            # 读入一维音频序列
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=10)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save('beatDatabase_mfcc.npy', beat_database)
    
    return beat_database

In [4]:
def readCorpus(path):
    
    # 读入音乐节奏数据库
    all_data = np.load(path, allow_pickle=True)
    beat_database = all_data.item()
    
    return beat_database

In [5]:
def updateCorpus(path, dbpath):
    
    # 音乐库位置
    audioList = os.listdir(path)
    
    # 已保存序列的文件
    raw_db = readCorpus(dbPath)
    raw_files = raw_db.keys()
    
    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav') and audioName not in raw_files:
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=10)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save(dbpath, beat_database)

In [6]:
def voiceCompare_quick(dbPath, tPath):
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # 读入要识别的录音
    y, sr = librosa.load(tPath)

    # 识别录音的节奏序列
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    beat_frames = librosa.feature.delta(beat_frames,mode ='nearest')
    x = array(beat_frames).reshape(-1, 1)

    # 将待识别的录音序列与语料库中语音逐一做DTW对比
    compare_result = {}
    
    for songID in beat_database.keys():
        y = beat_database[songID]
        y = array(y).reshape(-1, 1)
        
        dist = dtw(x, y).distance
        # print('两段话的差异程度为： ', songID.split("\\")[1], ": ", dist)
        
        compare_result[songID] = dist

    matched_song = min(compare_result, key=compare_result.get)
    print("最接近的录音是：", matched_song)

In [12]:
from sklearn import preprocessing

def voiceCompare(dbPath, tPath):
    # ==== 最大检索数 ====
    aimNum = 20
    # ==== DTW window size ====
    windowSize = 10
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # ==== 读入要识别的录音 ====
    y, sr = librosa.load(tPath)

    # 提取录音的 MFCC 特征
    x = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=10).T  # n1 * 10
    lenx = len(x)
    
    # 标准化
    for i in range(0, lenx):
        x[i] = preprocessing.minmax_scale(x[i])

    # ==== 将待识别的录音序列与语料库中语音逐一做DTW对比 ====
    
    # heap for [dist, 时间段，文件名]
    heap = []
    heapq.heapify(heap)  
    
    for songID in beat_database.keys():
        # 取出文件名对应的 mfcc 序列
        y = beat_database[songID].T
        
        leny = len(y) # n2 * 10 
        print(leny)
        
        # 标准化
        for i in range(0, leny):
            y[i] = preprocessing.minmax_scale(y[i])
        
        for tp in range(0, leny - lenx - windowSize):
            # *加速* 设定距离上限
            full = False  # 堆是否已满
            dist_UB = -10000  # DTW 距离上限
            overBound = False  # 是否过限
            
            if (len(heap) >= aimNum):
                full = True
                dist_UB = -heap[0][0]  # heap top (biggest) DTW dist as UB  
                
            # 计算 DTW(y[tp : tp + lenx + windowSize])
            total_dist = 0
            # euc_dist = 0
            
            for i in range(0, lenx):
                # DTW dist
                total_dist += dtw(x[i], y[tp + i], distance_only=False).distance
                # Euclidean dist
                # euc_dist += np.linalg.norm(p1 - p2)
                
                # *加速* 超过上限直接取消
                if (full and total_dist > dist_UB):
                    overBound = True
                    break
            
            # *加速* 超过上限
            if (overBound):
                continue
            
            # 入栈
            tupleY = (-total_dist, tp, songID) # dtw 距离加负数转为大根堆
            
            heapq.heappush(heap, tupleY)
            if (len(heap) > aimNum):
                heapq.heappop(heap)
            
            print(tupleY)
            
        # end for
        
        # 处理同名短间隔问题
        
        
    return heap

In [13]:
def getTimePoint(dbPath, tPath, vheap):
    res_num = 10 # 定义取出前 res_num 位的结果作为识别结果
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    # 提取前 res_num 个相似的片段并输出对应时间段
    similar_n = heapq.nlargest(res_num, vheap)
    
    print("开始输出相似片段：")
    
    for i in range(0, res_num):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数

        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")


In [14]:
# 语料库路径
corpus_path = './corpus'

# 数据表路径
dbPath = './beatDatabase_mfcc.npy';

# test file path
testPath = './input/00415250-前5s.wav'

In [15]:
# 1 初始化语料序列库
# beatDB = initialCorpus(corpus_path)

# 2 更新语料库中新音乐文件的序列
# updateCorpus(corpus_path, dbPath)

# 3 读入语料序列库
# beat_database = readCorpus(dbPath)

vheap = voiceCompare(dbPath, testPath)

3876
(-18.521133720874786, 0, './corpus\\00415250.wav')
(-60.46330001950264, 1, './corpus\\00415250.wav')
(-107.40511903166771, 2, './corpus\\00415250.wav')
(-146.32199355959892, 3, './corpus\\00415250.wav')
(-178.48492124676704, 4, './corpus\\00415250.wav')
(-201.94323900341988, 5, './corpus\\00415250.wav')
(-221.6008688211441, 6, './corpus\\00415250.wav')
(-236.75021243095398, 7, './corpus\\00415250.wav')
(-244.56108197569847, 8, './corpus\\00415250.wav')
(-253.09903821349144, 9, './corpus\\00415250.wav')
(-266.76910826563835, 10, './corpus\\00415250.wav')
(-278.121157258749, 11, './corpus\\00415250.wav')
(-285.2297625541687, 12, './corpus\\00415250.wav')
(-289.69937309622765, 13, './corpus\\00415250.wav')
(-292.60644868016243, 14, './corpus\\00415250.wav')
(-296.371660143137, 15, './corpus\\00415250.wav')
(-301.23861318826675, 16, './corpus\\00415250.wav')
(-304.2422043979168, 17, './corpus\\00415250.wav')
(-312.04654878377914, 18, './corpus\\00415250.wav')
(-323.30148819088936, 19,

In [16]:
getTimePoint(dbPath, testPath, vheap)

开始输出相似片段：
./corpus\00415250.wav , 0.00 秒, 5.00 秒
./corpus\00415250.wav , 0.02 秒, 5.02 秒
./corpus\00415250.wav , 0.05 秒, 5.05 秒
./corpus\00415250.wav , 0.07 秒, 5.07 秒
./corpus\00430105.wav , 115.04 秒, 120.04 秒
./corpus\00430105.wav , 115.06 秒, 120.06 秒
./corpus\00429126.wav , 156.15 秒, 161.15 秒
./corpus\00430105.wav , 115.09 秒, 120.09 秒
./corpus\00430105.wav , 115.02 秒, 120.02 秒
./corpus\00415250.wav , 0.09 秒, 5.09 秒


In [ ]:
inPara = sys.argv

if (len(inPara) < 2):
    print("请输入待识别录音文件路径！")
else:
    if (len(inPara) > 2):
        print("给定语料库路径为:", sys.argv[2])
        corpus_path = sys.argv[2]
    else:
        print("默认语料库路径为：", corpus_path)
    
    if (len(inPara) > 3):
        print("给定数据表路径为:", sys.argv[3])
        dbPath = sys.argv[3]
    else:
        print("默认数据表路径为：", dbPath)
    
    testPath = sys.argv[1]
    vheap = voiceCompare(dbPath, testPath)
    getTimePoint(dbPath, testPath, vheap)
  